In [13]:
%pip install yandex_chain python-dotenv
%pip install -U langchain-community

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [18]:
import os
from dotenv import load_dotenv
from yandex_chain import YandexLLM, YandexEmbeddings
from langchain.vectorstores import FAISS

load_dotenv()

True

In [19]:
# embeddings = YandexEmbeddings(folder_id="b1g8mvm4vfdho6fl7ole", api_key="AQVNy-2Y671NYKrRd0Ig_MMuKSw5L2DTyohvia6R")
# vectorstore = FAISS.from_texts(docs, embedding=embeddings)
# retriever = vectorstore.as_retriever()

LLM = YandexLLM(folder_id=os.getenv('FOLDER_ID'), api_key=os.getenv('YANDEX_API_KEY'))
print(LLM("How are you today?"))

Привет! Я — модель искусственного интеллекта, поэтому не могу сказать, как себя чувствую. Но могу предположить по текстовым данным о различных ситуациях, как закончила бы ответ на этот вопрос реальная личность.

Сегодня мой день проходит [так-то, вот так, следующим образом…].
